In [5]:
%config IPCompleter.greedy=True
%config Completer.use_jedi = False

In [18]:
# Download all the point cloud from the soruce website: run this once
import wget
with open("pointClouds/source.txt") as source:
    for line in source.readlines():
        name = line.strip()
        link = "https://people.sc.fsu.edu/~jburkardt/data/ply/" + name
        wget.download(link, out ="pointClouds/"+name )

In [1]:
# https://people.sc.fsu.edu/~jburkardt/data/ply/ply.html
# https://stackoverflow.com/questions/50965673/python-display-3d-point-cloud
%matplotlib inline
%config IPCompleter.greedy=True
%config Completer.use_jedi = False
import numpy as np
import open3d as o3d
import numpy.matlib
import scipy
import matplotlib.pyplot as plt

In [ ]:
pcd = o3d.io.read_point_cloud("airplane.ply") # Read the point cloud
print(pcd)
points = np.asarray(pcd.points)
# points = points[0:5, :]
print(points.shape)
o3d.visualization.draw_geometries([pcd])
fig, ax = plt.subplots(1,1, sharex=True, sharey=True)
# ax.scatter(points[:,0], points[:,1])
ones = np.ones((points.shape[0], 1))
points = np.concatenate((points, ones), axis=1)
P = np.array([[1., 0., 0., 0.],
[0., 1., 0., 0.],
[0., 0., 1., 0.]])
# Rotation matrix R is an orthonormal marix
R = np.array([[1., 0., 0.],
[0., 1., 0.],
[0., 0., 1.]])
K = np.array([[1., 0., 0.],
[0., 1., 0.],
[0., 0., 1.]])
t = np.array([[0.],
              1[0.],
              [-4000.]])
P1 = np.matmul(K, np.concatenate((R, t), axis=1))# P = K[R|t] = K[R|-RC]
R = np.array([[0., 1., 0.],
[1., 0., 0.],
[0., 0., 1.]])
K = np.array([[2., 0., 0.],
[0., 2., 0.],
[0., 0., 1.]])
P2 = np.matmul(K, np.concatenate((R, t), axis=1))
transfromed = np.matmul(P1, points.T).T
print(transfromed)
transfromed = transfromed/np.matlib.repmat(transfromed[:,2], 3, 1).T
ax.scatter(transfromed[:,0], transfromed[:,1])
transfromed = np.matmul(P2, points.T).T
transfromed = transfromed/np.matlib.repmat(transfromed[:,2], 3, 1).T
ax.scatter(transfromed[:,0], transfromed[:,1])
ax.axis('equal')
plt.show()

 np.set_printoptions(precision=4)
print('P1\n', P1)
print('P2\n', P2)

: # RQ Decomposition
# http://ksimek.github.io/2012/08/14/decompose/
def rq(M):
Q, R = np.linalg.qr(np.flipud(M).T)
# print(Q)
# print(R)
R = np.flipud(R.T)
R = np.fliplr(R)
Q = Q.T;
Q = np.flipud(Q)
return R, Q

In [ ]:
# P = [M|-MC]
# P = K[R|-RC]
M = P2[:, 0:3]
C = np.linalg.inv(M)@P1[:,3] # camera center in world coordinates
K, R = rq(M)
K, R = scipy.linalg.rq(M)
# make diagonal of K positive
T = np.diag(np.sign(np.diag(K)));
K = K @ T;
R = T @ R; # (T is its own inverse)
print(K)
print(R)

In [ ]:
 # make diagonal of K positive
T = np.diag(np.sign(np.diag(K)));
K = K @ T;
R = T @ R; # (T is its own inverse)
print(K)
print(R)

In [ ]:
# P from Hartley and Zisserman Example 6.2
Phz = np.array(
[[3.53553e+2, 3.39645e+2, 2.77744e+2, -1.44946e+6],
[-1.03528e+2, 2.33212e+1, 4.59607e+2, -6.32525e+5],
[7.07107e-1, -3.53553e-1, 6.12372e-1, -9.18559e+2]])

In [ ]:
# P = [M|-MC]
# P = K[R|-RC]
M = Phz[:, 0:3]
C = -np.linalg.inv(M)@Phz[:,3] # camera center in world coordinates
K, R = rq(M)
K, R = scipy.linalg.rq(M)
# make diagonal of K positive
T = np.diag(np.sign(np.diag(K)));
K = K @ T;
R = T @ R; # (T is its own inverse)
print(C)
print(K)
print(R)